In [1]:
import pandas as pd
import sqlite3
import contextlib 

sql_connect = sqlite3.connect('../database.sqlite')
cursor = sql_connect.cursor()

# SQL

Toon het aantal wedstrijden dat jouw team heeft gespeeld per seizoen.

Toon het aantal wedstrijden van jouw team in het kalenderjaar 2010 per seizoen.

Toon hoeveel punten ieder team in jouw competitie gehaald heeft per seizoen.

Toon op welke plaats jouw team daarmee is geëindigd in de ranglijst.

# Python

Maak een nieuw dataframe waarin het dataframe met onder andere het aantal punten per team wordt samengevoegd met het dataframe met daarin de teameigenschappen

Onderzoek in welke mate de teameigenschappen invloed hebben op het aantal punten. Maak hierbij gebruik van passende visualisaties.